In [1]:
#!/usr/bin/env python3.9
# coding=utf-8
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os
import requests as r

# muzete pridat libovolnou zakladni knihovnu ci knihovnu predstavenou na prednaskach
# dalsi knihovny pak na dotaz

""" Ukol 1:
načíst soubor nehod, který byl vytvořen z vašich dat. Neznámé integerové hodnoty byly mapovány na -1.

Úkoly:
- vytvořte sloupec date, který bude ve formátu data (berte v potaz pouze datum, tj sloupec p2a)
- vhodné sloupce zmenšete pomocí kategorických datových typů. Měli byste se dostat po 0.5 GB. Neměňte však na kategorický typ region (špatně by se vám pracovalo s figure-level funkcemi)
- implementujte funkci, která vypíše kompletní (hlubkou) velikost všech sloupců v DataFrame v paměti:
orig_size=X MB
new_size=X MB

Poznámka: zobrazujte na 1 desetinné místo (.1f) a počítejte, že 1 MB = 1e6 B. 
"""

' Ukol 1:\nnačíst soubor nehod, který byl vytvořen z vašich dat. Neznámé integerové hodnoty byly mapovány na -1.\n\nÚkoly:\n- vytvořte sloupec date, který bude ve formátu data (berte v potaz pouze datum, tj sloupec p2a)\n- vhodné sloupce zmenšete pomocí kategorických datových typů. Měli byste se dostat po 0.5 GB. Neměňte však na kategorický typ region (špatně by se vám pracovalo s figure-level funkcemi)\n- implementujte funkci, která vypíše kompletní (hlubkou) velikost všech sloupců v DataFrame v paměti:\norig_size=X MB\nnew_size=X MB\n\nPoznámka: zobrazujte na 1 desetinné místo (.1f) a počítejte, že 1 MB = 1e6 B. \n'

In [2]:
def _download_accidents(url: str = "http://ehw.fit.vutbr.cz/izv/accidents.pkl.gz", fname: str = "accidents.pkl.gz"):
    if not os.path.isfile(fname):
                    with r.get(url) as zips, open(fname, 'wb') as fp:
                        fp.write(zips.content)


In [3]:
def get_dataframe(filename: str, verbose: bool = False) -> pd.DataFrame:
    df = pd.read_pickle(filename)
    if verbose:
        mem = df.memory_usage(deep=True).sum()/ (1024^2)
        print(f"orig_size: {mem:.1f} MB",)
        
    convertable = ["p36", "p37", "weekday(p2a)", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13a",
               "p13b", "p13c", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23", "p24", "p27",
               "p28",
               "p34", "p35", "p39", "p44", "p45a", "p47", "p48a", "p49", "p50a", "p50b", "p51", "p52", "p53", "p55a",
               "p57", "p58", "p5a"]
    df['p2a'] = df['p2a'].astype('datetime64')
    df['date'] = df['p2a']
        #df = df.drop(columns=['p2a'])
    df[convertable] = df[convertable].astype('category')
    
    if verbose:
        mem = df.memory_usage(deep=True).sum()/ (1024^2)
        print(f"new_size:  {mem:.1f} MB",)
    return df

In [4]:
# Ukol 2: počty nehod v jednotlivých regionech podle druhu silnic

def plot_roadtype(df: pd.DataFrame, fig_location: str = None,
                  show_figure: bool = False):
    pass


In [5]:
# Ukol3: zavinění zvěří
def plot_animals(df: pd.DataFrame, fig_location: str = None,
                 show_figure: bool = False):
    pass


In [6]:
# Ukol 4: Povětrnostní podmínky
def plot_conditions(df: pd.DataFrame, fig_location: str = None,
                    show_figure: bool = False):
    pass

In [7]:
if __name__ == "__main__":
    _download_accidents()
    df = get_dataframe("accidents.pkl.gz", verbose=True)
    plot_roadtype(df, fig_location="01_roadtype.png", show_figure=True)
    plot_animals(df, "02_animals.png", True)
    plot_conditions(df, "03_conditions.png", True)


orig_size: 733436.4 MB
new_size:  704957.2 MB
new_size:  700489.9 MB
new_size:  534509.0 MB


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572934 entries, 0 to 572933
Data columns (total 65 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   p1            572934 non-null  object        
 1   p36           572934 non-null  category      
 2   p37           572934 non-null  category      
 3   weekday(p2a)  572934 non-null  category      
 4   p2b           572934 non-null  int64         
 5   p6            572934 non-null  category      
 6   p7            572934 non-null  category      
 7   p8            572934 non-null  category      
 8   p9            572934 non-null  category      
 9   p10           572934 non-null  category      
 10  p11           572934 non-null  category      
 11  p12           572934 non-null  category      
 12  p13a          572934 non-null  category      
 13  p13b          572934 non-null  category      
 14  p13c          572934 non-null  category      
 15  p14           572